In [1]:
!pip3 install torch torchvision torchaudio
!pip3 install transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from transformers import pipeline
from tqdm import tqdm
import torch
import json
import requests
import textwrap

You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.


/Users/arush/workspace/646proj/LaMP/data/flant5/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
with open("/Users/arush/workspace/646proj/LaMP/data/sampled_categorization_20.json", "r") as f:
    json_data = json.load(f)

In [3]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model_id = "google/flan-t5-xl"
summarizer = pipeline("summarization", model = model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
i = 0
golds = []

for user in tqdm(json_data):
    i += 1
    corpus = []
    categories = []
    profile = user['profile']
    query = user['input']

    input_prompt = "Your task is to answer the query based on user's relevant history interactions.\nHere’s the query: {}\n".format(query)
    past_interactions = []
    all_categories = ["women", "religion", "politics", "style & beauty", "entertainment", "culture & arts", "sports", "science & technology", 
    "travel", "business", "crime", "education", "healthy living", "parents", "food & drink"]
    news_with_categories = {str(i): [] for i in all_categories}  

    for doc in profile:
        corpus.append(doc['text'])
        categories.append(doc['category'])

    corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5, score_function=util.dot_score)
    hits = hits[0]
    predictions = {}

    for hit in hits:
        category = str(categories[hit['corpus_id']]) 
        news_with_categories[category].append(corpus[hit['corpus_id']])

    word_limit = 512
    for key, value in news_with_categories.items():
        all_news = ""
        if value:
            sentences_str = " ".join(value)
            words_so_far = len(all_news.split())
            words_in_current = len(sentences_str.split())
    
            if words_so_far + words_in_current <= word_limit:
                all_news += sentences_str + " "
            else:
                break

            final_result = " ".join(all_news.split()[:word_limit])
            input_prompt += "News category given by the user for the following news: '{}' is '{}'".format(summarizer(final_result)[0]['summary_text'], key) + "\n"
        else:
            break
            
    inputs = tokenizer(input_prompt, return_tensors="pt")
    outputs = llm_model.generate(**inputs)
    predictions["output"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    predictions["id"] = user["id"]
    golds.append(predictions)

final_json = {}
final_json["task"] = "LaMP_2"
final_json["golds"] = golds

  0%|                                                                                                                                                                                          | 0/20 [00:00<?, ?it/s]Your max_length is set to 200, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
/Users/arush/workspace/646proj/LaMP/data/flant5/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [04:04<00:00, 12.25s/it]


In [9]:
with open("pred_summarized_profile_news_categorization.json", "w") as f:
    json.dump(final_json, f)